In [17]:
!pip install google_play_scraper

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [19]:
apps_ids = ['br.com.light.ava', 'com.deway.ampla', 'br.com.brainweb.ifood', 'com.cerveceriamodelo.modelonow',

'com.mcdo.mcdonalds', 'habibs.alphacode.com.br',

'com.xiaojukeji.didi.brazil.customer',

'com.ubercab.eats', 'com.grability.rappi',

'burgerking.com.br.appandroid', 'br.com.alphacode.madero',

'com.vanuatu.aiqfome']

In [20]:
app_infos = []

for ap in tqdm(apps_ids):
    info = app(ap, lang='en', country='us')
    del info['comments']
    app_infos.append(info)

100%|██████████| 12/12 [00:04<00:00,  2.89it/s]


In [21]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.head()

,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,reviews,...,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,appId,url
0,Light Clientes,The Light Clientes app has a new look and new ...,The Light Clientes app has a new look and new ...,"Service channel of Light, power company of RJ","1,000,000+",1000000,1637572,2.866667,33399,34,...,None,Everyone,None,False,False,"Sep 28, 2015",1645482405,3.4,br.com.light.ava,https://play.google.com/store/apps/details?id=...
1,Enel Rio,Discover the Enel Rio App. Everything you need...,Discover the Enel Rio App. Everything you need...,Discover the Enel Rio app. Everything you need...,"1,000,000+",1000000,1442163,0.000000,0,0,...,None,Everyone,None,False,False,None,1691132606,10.30,com.deway.ampla,https://play.google.com/store/apps/details?id=...
2,iFood comida e mercado em casa,"Buy in markets, restaurants, pharmacies and pe...","Buy in markets, restaurants, pharmacies and pe...","Order food delivery, market, drinks, pharmacy ...","100,000,000+",100000000,104596475,4.651672,12331299,2601,...,None,Everyone,None,False,False,"Apr 25, 2012",1692023946,None,br.com.brainweb.ifood,https://play.google.com/store/apps/details?id=...
3,Zé Delivery de Bebidas,Zé Delivery: the biggest drinks app in the cou...,Zé Delivery: the biggest drinks app in the cou...,"Delivery of drinks at a low price, with fast d...","10,000,000+",10000000,19404602,4.638656,988068,287,...,https://play-lh.googleusercontent.com/Q5vqyqVe...,Mature 17+,None,False,False,"Aug 19, 2016",1691160637,23.30.1,com.cerveceriamodelo.modelonow,https://play.google.com/store/apps/details?id=...
4,McDonald's Offers and Delivery,Enter the new McDonald's App and get exclusive...,Enter the new McDonald&#39;s App and get exclu...,"Enjoy esclusive discounts, promotions and coup...","50,000,000+",50000000,66982092,4.500000,1198990,3156,...,None,Everyone,None,True,True,"Mar 27, 2017",1692280459,3.30.1,com.mcdo.mcdonalds,https://play.google.com/store/apps/details?id=...


In [22]:
app_reviews = []

for ap in tqdm(apps_ids):
    for score in list(range(1, 6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='pt',
                country='br',
                sort=sort_order,
                count= 1000 if score == 3 else 500,
                filter_score_with=score
            )
            for r in rvs:
                r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
                r['appId'] = ap
            app_reviews.extend(rvs)

100%|██████████| 12/12 [02:22<00:00, 11.84s/it]


In [23]:
len(app_reviews)

66456

In [24]:
app_reviews_df = pd.DataFrame(app_reviews)

In [25]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
0,145aacbc-c5e4-47c5-85f5-e253f1f319f2,Filipe Oliveira,https://play-lh.googleusercontent.com/a-/AD_cM...,Experiência horrível. O app parou de funcionar...,1,291,0.107.1,2022-02-05 12:40:00,None,NaT,0.107.1,most_relevant,br.com.light.ava
1,c3e1a6f1-c80b-44f2-ba88-a129202643b2,Renata Nascimento,https://play-lh.googleusercontent.com/a-/AD_cM...,"Bom dia, o app esta horrível, sempre usei era ...",1,278,1.25,2020-06-18 09:30:14,None,NaT,1.25,most_relevant,br.com.light.ava
2,c7562ba4-d59e-49b6-83d7-7dd1b9182ca7,Eduardo Junior,https://play-lh.googleusercontent.com/a-/AD_cM...,"Layout bonito, mas só! No momento em que você ...",1,216,0.107.1,2022-01-04 19:16:25,"Oi, Eduardo!\n\nObrigada pela sugestão de melh...",2022-01-10 14:42:39,0.107.1,most_relevant,br.com.light.ava
3,e1cbaf6c-6a32-4ba1-bdd3-160d795b80da,Neide Silva,https://play-lh.googleusercontent.com/a/AAcHTt...,Aplicativo horrível! Uma empresa séria que se ...,1,2,3.4,2023-05-28 13:21:09,None,NaT,3.4,most_relevant,br.com.light.ava
4,308b8f74-7433-4b5f-85cc-e364917ca06b,Marcos Ivo,https://play-lh.googleusercontent.com/a-/AD_cM...,"Que app horrível!!! Não sai da tela inicial, f...",1,1,3.4,2023-07-11 15:45:31,None,NaT,3.4,most_relevant,br.com.light.ava


In [26]:
app_reviews_df.to_csv('reviews.csv', index=None, header=True)